## 1. Import the Libraries

In [138]:
import pandas as pd
import numpy as np
import re
import csv
from unidecode import unidecode
import recordlinkage

## 3. Make Record Pairs

### 3.1 Block Indexing for Record Pairs

In [115]:
#############   BLOCK INDEXING
def runBlock():  
    block = 0
    both = 0
    
    #Next, check for exact match in name
    blockName = recordlinkage.BlockIndex(on=['name'])
    blockNamePairs = blockName.index(dfA, dfB)
    if len(blockNamePairs) > 0:
         block += 1
    
    #Next, check for exact match in addr    
    blockAddr = recordlinkage.BlockIndex(on=['addr'])
    blockAddrPairs = blockAddr.index(dfA, dfB)
    if len(blockAddrPairs) > 0:
        block += 2
        
    #Next, check for exact match in name AND addr    
    blockNA = recordlinkage.BlockIndex(on=['name','addr'])
    blockNAPairs = blockNA.index(dfA, dfB)
    if len(blockNAPairs) > 0:
        both = 1 
    
    if both == 1:
    #grab the id and contents of addr match
        return(returnResultsMI(blockNAPairs))
    else:
        
        if block == 0:
        #run sorted neighborhood both
            runSort()
        if block == 1:
        #grab the id and contents of name match
            if len(blockNamePairs) > 1:
                runCompName(blockNamePairs)
            else:
                return(returnResultsMI(blockNamePairs))        
        if block == 2:
            #grab the id and contents of addr match
            if len(blockAddrPairs) > 1:
                runCompAddr(blockAddrPairs)
            else:
                return(returnResultsMI(blockAddrPairs))
        if block == 3:
            #matches both, inconclusive
            runSort()
        

###############

### 3.2 Sorted Neighborhood Indexing

In [116]:
#############   SORTED NEIGHBORHOOD INDEXING
def runSort():
    sort = 0

    sortedNameIndexer = recordlinkage.SortedNeighbourhoodIndex(on='name')
    sortedNamePairs = sortedNameIndexer.index(dfA, dfB)
    if len(sortedNamePairs) > 0:
        sort += 1

    sortedAddrIndexer = recordlinkage.SortedNeighbourhoodIndex(on='addr')
    sortedAddrPairs = sortedAddrIndexer.index(dfA, dfB)
    if len(sortedAddrPairs) > 0:
        sort += 2

    if sort == 0:
        #run sorted neighborhood both
        runFull()
    if sort == 1:
        #compare with name pairs
        runCompName(sortedNamePairs)
    if sort == 2:
        #compare with addr pairs
        runCompAddr(sortedAddrPairs)
    if sort == 3:
        #compare with both pairs?
        runCompBoth(sortedNamePairs, sortedAddrPairs)

### 4. Compare Record Pairs

In [117]:
def runCompBoth(name_pairs,addr_pairs):
    compare = recordlinkage.Compare()

    compare.string('name', 'name', method='jarowinkler', threshold=0.95)
    compare.string('addr', 'addr', method='jarowinkler', threshold=0.95)
    compare.exact('city', 'city')
    compare.exact('ctry', 'ctry')
    compare.string('code', 'code', method='jarowinkler', threshold=0.90)
    
    nameMatch = False
    addrMatch = False
    # The comparison vectors for name
    featuresName = compare.compute(name_pairs, dfA, dfB)
    
    # The comparison vectors for addr
    featuresAddr = compare.compute(addr_pairs, dfA, dfB)

    ########## Classification

    featuresName.sum(axis=1).value_counts().sort_index(ascending=False)
    
    matchesNameAll = featuresName[featuresName.sum(axis=1) > 4]
    if len(matchesNameAll) > 0:
        nameMatch = True 
        matchesName = matchesNameAll #overwriting the larger set of results
    else:
        matchesName = featuresName[featuresName.sum(axis=1) > 3]
        if len(matchesName) > 0:
            nameMatch = True


    featuresAddr.sum(axis=1).value_counts().sort_index(ascending=False)
    
    matchesAddrAll = featuresAddr[featuresAddr.sum(axis=1) > 4]
    if len(matchesAddrAll) > 0:
        addrMatch = True 
        matchesAddr = matchesAddrAll #overwriting the larger set of results
    else:
        matchesAddr = featuresAddr[featuresAddr.sum(axis=1) > 3]
        if len(matchesAddr) > 0:
            addrMatch = True
    
########## FSM

    if (not nameMatch):
        if (not addrMatch):
            #run sorted neighborhood both
            runFull()
        elif (addrMatch):
            #grab the id and contents of addr match, 11 = preference towards addr
            if len(matchesAddr) > 2:
                runFull()
            else:
                return(returnResultsDF(matchesAddr))
    if (nameMatch):
        if(not addrMatch):
            #grab the id and contents of name match
            if len(matchesName) > 2:
                runFull()
            else:            
                return(returnResultsDF(matchesName))
        elif(addrMatch): 
            if len(matchesName) == 1:
                return(returnResultsDF(matchesName))
            elif len(matchesAddr) == 1:
                return(returnResultsDF(matchesAddr))
            else:
                runFull()

In [118]:
######### Specify to neighborhood & Compare
def runCompName(name_pairs):
    
    compare = recordlinkage.Compare()

    compare.string('name', 'name', method='jarowinkler', threshold=0.95)
    compare.string('addr', 'addr', method='jarowinkler', threshold=0.95)
    compare.exact('city', 'city')
    compare.exact('ctry', 'ctry')
    compare.string('code', 'code', method='jarowinkler', threshold=0.90)
    
    nameMatch = False
    addrMatch = False
    # The comparison vectors for name
    featuresName = compare.compute(name_pairs, dfA, dfB)

    ########## Classification

    featuresName.sum(axis=1).value_counts().sort_index(ascending=False)
    
    matchesNameAll = featuresName[featuresName.sum(axis=1) > 4]
    if len(matchesNameAll) > 0:
        nameMatch = True 
        matchesName = matchesNameAll #overwriting the larger set of results
    else:
        matchesName = featuresName[featuresName.sum(axis=1) > 3]
        if len(matchesName) > 0:
            nameMatch = True

########## FSM

    if (not nameMatch):
        if (not addrMatch):
            #run sorted neighborhood both
            runFull()
        elif (addrMatch):
            #grab the id and contents of addr match, 11 = preference towards addr
            if len(matchesAddr) > 2:
                runFull()
            else:
                return(returnResultsDF(matchesAddr))
    if (nameMatch):
        if(not addrMatch):
            #grab the id and contents of name match
            if len(matchesName) > 2:
                runFull()
            else:            
                return(returnResultsDF(matchesName))
        elif(addrMatch): 
            #grab the id and contents of addr match, 11 = preference towards addr
            runFull()

In [119]:
######### Specify to neighborhood & Compare
def runCompAddr(addr_pairs):
    
    compare = recordlinkage.Compare()

    compare.string('name', 'name', method='jarowinkler', threshold=0.95)
    compare.string('addr', 'addr', method='jarowinkler', threshold=0.95)
    compare.exact('city', 'city')
    compare.exact('ctry', 'ctry')
    compare.string('code', 'code', method='jarowinkler', threshold=0.90)
    
    nameMatch = False
    addrMatch = False
    # The comparison vectors for addr
    featuresAddr = compare.compute(addr_pairs, dfA, dfB)

    ########## Classification

    featuresAddr.sum(axis=1).value_counts().sort_index(ascending=False)
    
    matchesAddrAll = featuresAddr[featuresAddr.sum(axis=1) > 4]
    if len(matchesAddrAll) > 0:
        addrMatch = True 
        matchesAddr = matchesAddrAll #overwriting the larger set of results
    else:
        matchesAddr = featuresAddr[featuresAddr.sum(axis=1) > 3]
        if len(matchesAddr) > 0:
            addrMatch = True
    
########## FSM
    if (not nameMatch):
        if (not addrMatch):
            #run sorted neighborhood both
            runFull()
        elif (addrMatch):
            #grab the id and contents of addr match, 11 = preference towards addr
            if len(matchesAddr) > 2:
                runFull()
            else:
                return(returnResultsDF(matchesAddr))
    if (nameMatch):
        if(not addrMatch):
            #grab the id and contents of name match
            if len(matchesName) > 2:
                runFull()
            else:            
                return(returnResultsDF(matchesName))
        elif(addrMatch): 
            #grab the id and contents of addr match, 11 = preference towards addr
            runFull()

### 5. Classification and Matching

### 6. Full Index Pairs and Classification

In [120]:
####### If nothing else finds matches, run FULL INDEX
def runFull():
    compare = recordlinkage.Compare()

    compare.string('name', 'name', method='jarowinkler', threshold=0.90)
    compare.string('addr', 'addr', method='jarowinkler', threshold=0.95)
    compare.exact('city', 'city')
    compare.exact('ctry', 'ctry')
    compare.string('code', 'code', method='jarowinkler', threshold=0.90)
    
    fullIndexer = recordlinkage.FullIndex()
    fullIndexPairs = fullIndexer.index(dfA, dfB)

    featuresFull = compare.compute(fullIndexPairs, dfA, dfB)

    matchesFullAll = featuresFull[featuresFull.sum(axis=1) > 4]
    if len(matchesFullAll) > 0:
        return(returnResultsDF(matchesFullAll))
    else:
        matchesFull = featuresFull[featuresFull.sum(axis=1) > 3]
        if len(matchesFull) >0:
            return(returnResultsDF(matchesFull))
    #return the match/matches with highest sum. Maybe try >4 first then >3. for row in frame, 
    #grab id then return the full dict entry of the id
    

### 7. Return Results

In [121]:
######## RETURN FROM MULTIINDEX

def returnResultsMI(pairs):
    data = pairs.to_frame(index = False)[0]
    i = 0
    grab_ids = []
    while i < len(data):
        grab_ids.append(data[i])
        i+=1 
    for grab_id in grab_ids:
        result = dfD.loc[grab_id].to_string(header = False, index = False)
        results.append(result)
    return results

def returnResultsDF(pairs):
    pairs2 = pairs.index
    data = pairs2.to_frame(index = False)[0]
    i = 0
    grab_ids = []
    while i < len(data):
        grab_ids.append(data[i])
        i+=1   
    for grab_id in grab_ids:
        result = dfD.loc[grab_id].to_string(header = False, index = False)
        results.append(result)
    return results

### 8. Main Method to Run

In [122]:
# Do this later
### MAIN METHOD
# First, check for exact match overall FULL
# results = []
# blockIndexer = recordlinkage.BlockIndex(on=['name', 'addr', 'city', 'ctry', 'code'])
# blockIndexPairs = blockIndexer.index(dfA, dfB)
# if len(blockIndexPairs) <= 0:
#     runBlock()

### Now perform Edit distance if multiple results

In [123]:
INSERTION_PENALTY = 1
DELETION_PENALTY = 1
# This substitution penalty differentiates from Levenshtein cost (would be 1)
SUBSTITUTION_PENALTY = 2
ALLOWED_LEVELS = ["word", "char"]
LEVEL = "word"

In [124]:
def compute_cost(D, i, j, token_X, token_Y):
    relative_subst_cost = 0 if token_X == token_Y else SUBSTITUTION_PENALTY
    return min(D[i-1, j] + INSERTION_PENALTY, D[i, j-1] + DELETION_PENALTY, D[i-1, j-1] + relative_subst_cost)

In [125]:
def tokenize_string(string, level="word"):
    assert level in ALLOWED_LEVELS
    if level is "word":
        return string.split(" ")
    else:
        return list(string)

In [126]:
def minimum_edit_distance(string1, string2, level="word"):
    """The function uses the dynamic programming approach from Wagner-Fischer to compute the minimum edit distance
    between two sequences.
    :param string1 first sequence
    :param string2 second sequence
    :param level defines on which granularity the algorithm will be applied. "word" specifies the token to
    be sequential words while "char" applies the algorithm on a character-by-character level"""
    # Call tokenize string on the two address strings that were passed to the method
    string1_tokens = tokenize_string(string1, level)
    string2_tokens = tokenize_string(string2, level)
    n = len(string1_tokens)
    m = len(string2_tokens) 
    D = np.zeros((n, m))
 
    for i in range(n):
        for j in range(m):
            if j == 0:
                D[i,j] = i
            elif i == 0:
                D[i,j] = j
            else:
                D[i,j] = compute_cost(D, i, j, string1_tokens[i], string2_tokens[j])
 
    return string2_tokens, D[n-1, m-1]

In [127]:
def preProcess(column):
    # convert any unicode data into ASCII characters
    column = unidecode(column)
    # ignore new lines
    column = re.sub('\n', ' ', column)
    # ignore special characters
    column = re.sub('-', '', column)
    column = re.sub('/', ' ', column)
    column = re.sub("'", '', column)
    column = re.sub(",", '', column)
    column = re.sub(":", ' ', column)
    # ignore extra white space
    column = re.sub('  +', ' ', column)
    # ignore casing
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column

### Collect user entry and run against record linkage

In [128]:
name = "1 mobile LIMITED"
addr = "30 CITY ROADS"
city = "LONDON"
ctry="UK"
code="EC1Y 2AB"
with open ('user_input_file.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'name', 'addr', 'city', 'ctry', 'code']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'id' : "1", 'name': name, 'addr': addr, 'city':city, 'ctry':ctry, 'code':code})
csvfile.close()

dfA = pd.read_csv("companies_final.csv")
dfB = pd.read_csv('user_input_file.csv')
dfD = pd.read_csv("companies_dict.csv")
dfA.drop('id', axis=1)
dfB.drop('id', axis=1)

results = []
blockIndexer = recordlinkage.BlockIndex(on=['name', 'addr', 'city', 'ctry', 'code'])
blockIndexPairs = blockIndexer.index(dfA, dfB)
if len(blockIndexPairs) <= 0:
    runBlock()

### Output to User

In [133]:
def output(results):
    response_rl = "No matching address was found!"
    if len(results) == 1:
        response_rl = results[0]
    user_entry = name + " " + addr + " " + city + " " + ctry + " " + code 
    user_proc = preProcess(user_entry)
    min_dist = 9999
    if 1 < len(results) < 4:
        for result in results:
            result_proc = preProcess(result)
            dist = minimum_edit_distance(result_proc, user_proc)[1]
            if (dist < min_dist):
                min_dist = dist
                response_rl = result
    print("User entry:", user_entry)
    print("Response:", response_rl)
    return ""

### Demo Examples

In [157]:
name = "1 MOBile LIMITED"
addr = "30 CITY ROADS"
city = "LONDON"
ctry="UK"
code="EC1Y 2AB"
with open ('user_input_file.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'name', 'addr', 'city', 'ctry', 'code']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'id' : "1", 'name': name, 'addr': addr, 'city':city, 'ctry':ctry, 'code':code})
csvfile.close()

dfA = pd.read_csv("companies_final.csv")
dfB = pd.read_csv('user_input_file.csv')
dfD = pd.read_csv("companies_dict.csv")
dfA.drop('id', axis=1)
dfB.drop('id', axis=1)

results = []
blockIndexer = recordlinkage.BlockIndex(on=['name', 'addr', 'city', 'ctry', 'code'])
blockIndexPairs = blockIndexer.index(dfA, dfB)
if len(blockIndexPairs) <= 0:
    runBlock()

In [174]:
name = "1 MOBILE LIMITED"
addr = "400 Center Pointe Lane"
city = "LONDON"
ctry="UK"
code="EC1Y 2AB"
with open ('user_input_file2.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'name', 'addr', 'city', 'ctry', 'code']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'id' : "1", 'name': name, 'addr': addr, 'city':city, 'ctry':ctry, 'code':code})
csvfile.close()

dfA = pd.read_csv("companies_final.csv")
dfB = pd.read_csv('user_input_file2.csv')
dfD = pd.read_csv("companies_dict.csv")
dfA.drop('id', axis=1)
dfB.drop('id', axis=1)

results = []
blockIndexer = recordlinkage.BlockIndex(on=['name', 'addr', 'city', 'ctry', 'code'])
blockIndexPairs = blockIndexer.index(dfA, dfB)
if len(blockIndexPairs) <= 0:
    runBlock()

In [192]:
name = ""
addr = "COURTYARD SUITE 100 HATTON GARDEN"
city = "LONDON"
ctry="UK"
code="EC1N 8NX"
with open ('user_input_file2.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'name', 'addr', 'city', 'ctry', 'code']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'id' : "1", 'name': name, 'addr': addr, 'city':city, 'ctry':ctry, 'code':code})
csvfile.close()

dfA = pd.read_csv("companies_final.csv")
dfB = pd.read_csv('user_input_file2.csv')
dfD = pd.read_csv("companies_dict.csv")
dfA.drop('id', axis=1)
dfB.drop('id', axis=1)

results = []
blockIndexer = recordlinkage.BlockIndex(on=['name', 'addr', 'city', 'ctry', 'code'])
blockIndexPairs = blockIndexer.index(dfA, dfB)
if len(blockIndexPairs) <= 0:
    runBlock()

# User Queries

### Partial Case Matching

In [158]:
print(output(results))

User entry: 1 MOBile LIMITED 30 CITY ROADS LONDON UK EC1Y 2AB
Response: 1 MOBILE LIMITED 30 CITY ROAD LONDON EC1Y 2AB



### Incorrect Street Name

In [175]:
print(output(results))

User entry: 1 MOBILE LIMITED 400 Center Pointe Lane LONDON UK EC1Y 2AB
Response: 1 MOBILE LIMITED 30 CITY ROAD LONDON EC1Y 2AB



### Aliases and Whitespace

In [191]:
print(output(results))

User entry: 1 MOB        LTD 30 CITY ROADS LONDON UK EC1Y 2AB
Response: 1 MOBILE LIMITED 30 CITY ROAD LONDON EC1Y 2AB



### Exact Unique Addresses

In [193]:
print(output(results))

User entry:  COURTYARD SUITE 100 HATTON GARDEN LONDON UK EC1N 8NX
Response: ACTURIS LIMITED COURTYARD SUITE 100 HATTON GAR...

